# Laser Sintering Process Parameter Investigation

||
|--------|-------------|
| Author: | Peter Denno |
| Creation Date:   | 2014-08-12  |
|Part Number: | 1234a|
|Report ID: | 725A27D7-D794-408A-9409-F8820A246D53|


## Introduction

This report develops a predictive model for the selective laser sintering of production of part 1234a. (See process task xyz). The analysis described in this report picks up after screening experiments have identified that layer thickness, scan speed and oxygen content are key factors.  

*POD ToDo: Hide this detail*

In [1]:
import numpy as np
import statsmodels.api as sm # was up-to-date already (Latest version is 0.50 from August, 2013.)
import matplotlib.pyplot as plt
import pyDOE as doe  # "pip install --upgrade pyDOE" for this.
import pandas as pd # "pip install --upgrade pandas" got version 0.14.1 (July, 2014, most-up-to-date stable version)
from IPython.core.display import * # This gave me the Greek alphabet in MathJax, I think.
from patsy import dmatrices # This is for creating design matrices using formulas similar to R.
%matplotlib inline 
%aimport ereps

The first time you execute the next statement, it creates information about the report in the engineering report ontology. Every other time you execute it, it returns some of the information it originally created. * [Technical description: the http server starts a session (with a session cookie) and links all objects created in the report to that report. ] *

In [2]:
report = ereps.register_report("725A27D7-D794-408A-9409-F8820A246D53")

ConnectionError: HTTPConnectionPool(host='localhost', port=3010): Max retries exceeded with url: /modelegator/modelica/start-session?report-uuid=725A27D7-D794-408A-9409-F8820A246D53 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x11c4a22b0>: Failed to establish a new connection: [Errno 61] Connection refused',))

## Box-Behnken design

In [3]:
bbarray = doe.bbdesign(3) # Three factor Box-Behnken Design matrix
bbarray

/Users/pdenno/anaconda/lib/python3.5/site-packages/pyDOE/doe_factorial.py:78: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  rng = lvl*range_repeat
/Users/pdenno/anaconda/lib/python3.5/site-packages/pyDOE/doe_repeat_center.py:43: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return np.zeros((repeat, n))


array([[-1., -1.,  0.],
       [ 1., -1.,  0.],
       [-1.,  1.,  0.],
       [ 1.,  1.,  0.],
       [-1.,  0., -1.],
       [ 1.,  0., -1.],
       [-1.,  0.,  1.],
       [ 1.,  0.,  1.],
       [ 0., -1., -1.],
       [ 0.,  1., -1.],
       [ 0., -1.,  1.],
       [ 0.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

Normalized explanatory variables:
* $x_1$ = Scanning Speed (50, 100, 150) $mm/s$
* $x_2$ = Layer Thickness (20, 35, 50) $\mu m$ 
* $x_3$ = Oxygen content (.13, .15, .17) 

The variables are normalized so that they each vary over [-1,1]. Thus:
* $X_1 = (x_1-100) / 50$
* $X_2 = (x_2 - 35) / 15$
* $X_3 = (x_3 - 0.15) / .02$

### Get density data from Box-Behken experiments

We reference experimental data through Jobs, which are occurrences of ProcessPlans. A TaskOccurrence in each job performs the density measurement. The data is ordered to correspond to the experimental runs using bbarray. 

In [4]:
density = ereps.density_data(plans=ereps.job_list('report1',order_by=bbarray),task=2)
density

ConnectionError: HTTPConnectionPool(host='localhost', port=8083): Max retries exceeded with url: /tbl/sparqlmotion?id=getDensity1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x11c55f438>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [5]:
(len(density), len(bbarray)) # Make sure I have all the data.

NameError: name 'density' is not defined

### Part density quadratic model, no 3-factor interaction

$Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \beta_3X_3 + \beta_{12}X_1X_2 + \beta_{13}X_1X_3 + \beta_{23}X_2X_3
 + \beta_{11}X_1^2 + \beta_{22}X_2^2 + \beta_{33}X_3^2 + experimental \ error$

Register such an equation for density response (Yden).

In [6]:
eqn = ereps.register_predictive("Yden = b0 + b1*x1 + b2*x2 + b3*x3 + b12*x1*x2 + b13*x1*x3 + b23*x2*x3 + b11*x1^2 + b22*x2^2 + b33*x3^2")

ConnectionError: HTTPConnectionPool(host='localhost', port=3010): Max retries exceeded with url: /modelegator/modelica/register-eqn/?eqn=Yden+%3D+b0+%2B+b1%2Ax1+%2B+b2%2Ax2+%2B+b3%2Ax3+%2B+b12%2Ax1%2Ax2+%2B+b13%2Ax1%2Ax3+%2B+b23%2Ax2%2Ax3+%2B+b11%2Ax1%5E2+%2B+b22%2Ax2%5E2+%2B+b33%2Ax3%5E2 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x11c5774a8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [7]:
ereps.register_var(var=eqn+':x1',property_name='scan_speed')
ereps.register_var(var=eqn+':x2',property_name='layer_thickness')
ereps.register_var(var=eqn+':x3',property_name='oxygen_content')
ereps.register_var(var=eqn+':Yden',property_name='density')

NameError: name 'eqn' is not defined

### Register the intended sense of variables.

Here the engineer goes 'off line' to find URLs in process terms, material terms and dimensions ontologies.

In [8]:
ereps.register_var_sense(var=eqn+':x1',urls=['http://modelmeth.nist.gov/processTerms/am/laser_scan_speed'])
ereps.register_var_sense(var=eqn+':x2',urls=['http://modelmeth.nist.gov/processTerms/am/layer_thickness'])
ereps.register_var_sense(var=eqn+':x3',urls=['http://modelmeth.nist.gov/materialTerms/powder_met/oxygen_content'])
ereps.register_var_sense(var=eqn+':Yden',urls=['http://modelmeth.nist.gov/dimensions/density/archimedes_method'])


NameError: name 'eqn' is not defined

### Construct table for analysis.

In [9]:
df = pd.DataFrame(bbarray,columns=['scan_speed','layer_thickness','oxygen_content']) # assign variables to columns

In [10]:
df['density'] = density # add a column for the response variable data
df

NameError: name 'density' is not defined

'formula_text' below is in the syntax of a language called Patsy, which the statsmodels package uses to define design matrices. The term wrapped in \*\* 2 provides the design matrix with the 3 factors plus their two factor combinations. The terms wrapped in I(...) provide the second order factors.

In [11]:
formula_text = 'density ~ (scan_speed + layer_thickness + oxygen_content) ** 2 + I(scan_speed ** 2) + I(layer_thickness ** 2) + I(oxygen_content ** 2)'

In [12]:
y, X = dmatrices(formula_text, data=df, return_type='dataframe')

PatsyError: Error evaluating factor: NameError: name 'density' is not defined
    density ~ (scan_speed + layer_thickness + oxygen_content) ** 2 + I(scan_speed ** 2) + I(layer_thickness ** 2) + I(oxygen_content ** 2)
    ^^^^^^^

### Perform Ordinary Least Squares (OLS) Fit

In [13]:
mod = sm.OLS(y, X)

NameError: name 'y' is not defined

In [14]:
density_fit = mod.fit()

NameError: name 'mod' is not defined

In [15]:
print(density_fit.summary())

NameError: name 'density_fit' is not defined

In [16]:
density_fit.params

NameError: name 'density_fit' is not defined

### Compare predicted with actual

In [17]:
def density_predict (scan_speed, layer_thick, oxygen):
    "predictive equation for density, from full quadratic model"
    den = density_fit.params[0] + \
    density_fit.params[1] * scan_speed + \
    density_fit.params[2] * layer_thick + \
    density_fit.params[3] * oxygen + \
    density_fit.params[4] * scan_speed * layer_thick + \
    density_fit.params[5] * scan_speed * oxygen + \
    density_fit.params[6] * layer_thick * oxygen + \
    density_fit.params[7] * scan_speed * scan_speed + \
    density_fit.params[8] * layer_thick * layer_thick + \
    density_fit.params[9] * oxygen * oxygen 
    return (den)

In [18]:
df['predicted'] = [density_predict(*bbarray[i]) for i in range(0,len(bbarray))]

NameError: name 'density_fit' is not defined

In [ ]:
df = df.rename(columns={'density' : 'actual'}) # rename so we don't get confused

In [ ]:
df['error'] = df.apply(lambda x : abs(x['actual']-x['predicted']) / x['actual'], axis=1)
df

In [ ]:
df['error'].sum()

### Plot results

*POD: In the following, make sure you have the dimensions correct. (Try different deltas. Compare with contour plot example.)*

In [ ]:
delta = 0.025
x_sspeed = np.arange(-1.0, 1.0, delta)
y_lthick = np.arange(-1.0, 1.0, delta)
z_density = np.array([[density_predict(x,y,0) for x in x_sspeed] for y in y_lthick]) # oxygen = 0
x_grid, y_grid = np.meshgrid(x_sspeed, y_lthick)

In [ ]:
plt.figure()
CS = plt.contour(x_grid, y_grid, z_density)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Predicted density: x=scan speed, y=layer thickness at oxygen=0')

### Save the completed equation

In [ ]:
ereps.assert_predictive('Eqn1',density_fit)